In [ ]:
# use optimazation to find the best quality
import torch
import os
import json
import dnnlib
import legacy
import numpy as np 
import tqdm
dataset_json = '/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/mirror_non_makeup/all_mt_dataset.json'
# input params
target_img_path = ''
num_steps = 10000

c_basename = os.path.basename(target_img_path)
network_pkl = '../pretrained_models/ffhq512-128.pkl'
device = torch.device('cuda')
# load eg3d decoder
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)
    G.eval()
# Compute w stats.
w_avg_samples = 1000
z_samples = np.random.RandomState(123).randn(w_avg_samples, G.z_dim)
w_samples = G.mapping(torch.from_numpy(z_samples).to(device), None)  # [N, L, C]
w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)  # [N, 1, C]
w_avg = np.mean(w_samples, axis=0, keepdims=True)  # [1, 1, C]
w_std = (np.sum((w_samples - w_avg) ** 2) / w_avg_samples) ** 0.5
start_w = w_avg


#load camera
with open(dataset_json, 'r') as f:
    camera_dic = dict(json.load(f)['labels'])

c = camera_dic[c_basename]

# init w_opt
w_opt = torch.tensor(start_w, dtype=torch.float32, device=device,
                         requires_grad=True)  # pylint: disable=not-callable

# init optimizer
optimizer = torch.optim.Adam([w_opt] , betas=(0.9, 0.999),
                                 lr=5e-3)


# give a latent
latent = torch.randn(1,14,512).to(device)
for step in tqdm(range(num_steps)):
    # Learning rate schedule.
    t = step / num_steps
    w_noise_scale = w_std * initial_noise_factor * max(0.0, 1.0 - t / noise_ramp_length) ** 2
    lr_ramp = min(1.0, (1.0 - t) / lr_rampdown_length)
    lr_ramp = 0.5 - 0.5 * np.cos(lr_ramp * np.pi)
    lr_ramp = lr_ramp * min(1.0, t / lr_rampup_length)
    lr = initial_learning_rate * lr_ramp
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
# go a generator

    img = G.synthesis(latent, c, noise_mode='const')['image']



# design loss


#optimaze the code 



